<a href="https://colab.research.google.com/github/fromakim/2021Election_Analysis/blob/main/topic_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Dev Environment

In [ ]:
!pip install tomotopy
!pip install hangul_utils

In [2]:
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git

  Cloning https://github.com/haven-jeon/PyKoSpacing.git to /tmp/pip-req-build-xowv1eeb
  Running command git clone -q https://github.com/haven-jeon/PyKoSpacing.git /tmp/pip-req-build-xowv1eeb
     |████████████████████████████████| 454.4 MB 23 kB/s 
     |████████████████████████████████| 4.0 MB 39.2 MB/s 
     |████████████████████████████████| 462 kB 55.9 MB/s 
     |████████████████████████████████| 1.2 MB 43.4 MB/s 
  Created wheel for pykospacing: filename=pykospacing-0.5-py3-none-any.whl size=2268581 sha256=40cbfa997cbad4075c1d70ed6e20fb296c1f987151ae2b44ef8efe372dcfc753
  Stored in directory: /tmp/pip-ephem-wheel-cache-frag6504/wheels/9b/93/81/a2a7dc8c66ede5bf30634d20635f32b95eac7ca2ea8844058b
  Created wheel for wrapt: filename=wrapt-1.12.1-cp37-cp37m-linux_x86_64.whl size=68720 sha256=299d582311daf069578a5d25edb96fe38cc89b8689cdb464e89a0e7ef85c447a
  Stored in directory: /root/.cache/pip/wheels/62/76/4c/aa25851149f3f6d9785f6c869387ad82b3fd37582fa8147ac6
Successfully built pyko

In [3]:
!git clone https://github.com/kakao/khaiii.git
!pip install cmake
!mkdir build
!cd build && cmake /content/khaiii
!cd /content/build/ && make all
!cd /content/build/ && make resource
!cd /content/build && make install
!cd /content/build && make package_python
!pip install /content/build/package_python

Cloning into 'khaiii'...
remote: Enumerating objects: 1020, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 1020 (delta 0), reused 0 (delta 0), pack-reused 1016
Receiving objects: 100% (1020/1020), 33.03 MiB | 23.70 MiB/s, done.
Resolving deltas: 100% (433/433), done.
-- [hunter] Initializing Hunter workspace (70287b1ffa810ee4e952052a9adff9b4856d0d54)
-- [hunter]   https://github.com/ruslo/hunter/archive/v0.23.34.tar.gz
-- [hunter]   -> /root/.hunter/_Base/Download/Hunter/0.23.34/70287b1
-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
--

In [4]:
import nltk
from nltk.corpus import stopwords
from pykospacing import Spacing
from khaiii import KhaiiiApi
from hangul_utils import split_syllables, join_jamos

In [5]:
import numpy as np
import pandas as pd
import tomotopy as tp

In [6]:
from google.colab import drive

In [7]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# 2. Import Data

In [8]:
tweets = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/election_sample/tweet.csv')

In [9]:
tweets['text']

0       윤석열이랑 이재명이랑 사랑의 도피해서 심상정이 대통령된 다음에 동성혼 법제화 시켜서...
1       [속보] 20대 대통령 선거 후보 확정\n더불어민주당 이재명\n국민의힘 윤석열\n정...
2       정확히 표현해야 합니다.\n\n- 이재명을 안찍는건 당연.\n- 기권을 하면 이재명...
3       이 심상정의 1분을 어떻게 잊을 수 있을까. 나는 절대 못 잊는다. https://...
4       심상정을 뽑는 표가 사표가 아니란걸 우리 여성들이 보여줘야 한다. 인구 절반의 여성...
                              ...                        
5316    윤석열은 끊임없이 까면서\n이재명에 대해 일언반구도 없는\n당신도 똑같은 역사의 죄...
5317    💚다음주 토요일 서초집회 아젠다💚\n📌대장동 특검\n📌합수부 설치\n📌이재명 구속\...
5318    아니 애초에 이재명이 민주당안밖에서 반문질로 지금까지 커온거 사실인데 무슨 문재인을...
5319    여기는 경북울진 주변에 많은 사람들이 공통적으로 하는말 “윤석렬 너무 수준떨어진다”...
5320    김종혁 전 국장 "정진상이 뭔가 고리가 돼 있거나 의심받고 있는데 떡 하니 부실장으...
Name: text, Length: 5321, dtype: object

In [10]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [11]:
stopwords = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/stopwords.csv')

# 3. Data Preprocessing

In [12]:
spacing = Spacing()
api = KhaiiiApi()

In [13]:
relations = ['JKS', 'JKC', 'JKG', 'JKO', 'JKB', 'JKV', 'JKQ', 'JX', 'JC']           # 관계언 (조사)
dependents = ['EP', 'EF', 'EC', 'ETN', 'ETM', 'XPN', 'XSN', 'XSV', 'XSA', 'XR']     # 의존격 (어미, 접사)
symbols = ['SF', 'SP', 'SS', 'SE', 'SO', 'SL', 'SH', 'SW', 'SWK', 'SN']             # 기호

In [14]:
unused = []
unused.extend(relations)
unused.extend(dependents)
unused.extend(symbols)

In [15]:
def flatten(list_of_lists):
    '''
    Return flattened list from list of lists.
    '''
    return [y for x in list_of_lists for y in x]

In [16]:
def khaiii_analyze(phrase):
    '''
    Return Lexicons analyzed by Khaiii Analyzer.
    '''
    return [z.lex for z in flatten([y.morphs for y in api.analyze(phrase)]) if z.tag not in unused]

In [17]:
tweets['original'] = tweets['text']

In [18]:
# Erase Non-Korean and Non-English Keywords
tweets['text'] = tweets['text'].str.replace('[^A-Za-z가-힣0-9ㄱ-ㅎㅏ-ㅣ.,/ ]', ' ')

In [19]:
tweets['text'] = tweets['text'].map(split_syllables)

In [20]:
tweets['text'] = tweets['text'].map(join_jamos)

In [21]:
tweets['text'] = tweets['text'].map(lambda x : x.replace(' ', ''))

In [22]:
tweets['text'] = tweets['text'].map(spacing)

In [23]:
tweets['text'] = tweets['text'].map(lambda x : ' '.join(khaiii_analyze(x) if x != '' else ''))

In [25]:
tweets['text'] = tweets['text'].apply(lambda x : [y for y in x.split(' ') if y not in stopwords])

In [26]:
tweets['text']

0       [윤석열, 이재명, 사랑, 도, 피하, 심상정, 대통령, 되, 다음, 동성혼법, 제...
1       [속보, 대, 대통령, 선거, 후보, 확정, 더불, 민주, 당이재, 명, 국민, 힘...
2       [정확히, 표현, 하, 이재명, 안, 찍, 것, 당연, 기권, 하, 이재명, 도와주...
3                   [이, 심상정, 분, 어떻, 잊, 수, 있, 나, 절대, 못, 잊]
4       [심상정, 뽑, 표, 사표, 아니, 것, 우리, 여성, 보이, 주, 하, 인구, 절...
                              ...                        
5316    [윤석열, 끊임없이, 까, 이재명, 대하, 일언반구, 없, 당신, 똑같, 역사, 죄...
5317    [다음, 주, 토요일, 서초집회, 아젠다대, 장, 동, 특검, 합수부, 설치, 이재...
5318    [아니, 애초, 이재명, 민주당, 안, 반문질, 지금, 커, 오, 거, 사실, 이,...
5319    [여기, 경북, 울진, 주변, 많, 사람, 공통, 하, 말, 윤석렬, 너무, 수준,...
5320    [김종혁, 전, 국장, 정진상, 뭐, 이, 고리, 되, 있, 의심, 받, 있, 떡,...
Name: text, Length: 5321, dtype: object

# 4. Run Model

In [90]:
model = tp.LDAModel(k = 10, rm_top = 20, min_cf = 20, alpha = 0.3, eta = 0.1)

In [91]:
for i in range(len(tweets)):
    if tweets['text'][i] != []:
        model.add_doc(tweets['text'][i])

In [92]:
for i in range(0, 1000, 100):
    model.train(100)
    print('Iteration: {}\tLog-likelihood: {}'.format(i, model.ll_per_word))

Iteration: 0	Log-likelihood: -6.111092894507371
Iteration: 100	Log-likelihood: -6.012142658044959
Iteration: 200	Log-likelihood: -5.963128008114813
Iteration: 300	Log-likelihood: -5.960792485770114
Iteration: 400	Log-likelihood: -5.949990666806325
Iteration: 500	Log-likelihood: -5.9525849002789855
Iteration: 600	Log-likelihood: -5.950211098694289
Iteration: 700	Log-likelihood: -5.946916056954921
Iteration: 800	Log-likelihood: -5.943523346903233
Iteration: 900	Log-likelihood: -5.938824955759266


In [93]:
for k in range(model.k):
    print(f'Top 10 words of topic #{k}')
    print(model.get_topic_words(k, top_n = 10))

Top 10 words of topic #0
[('말', 0.012337288819253445), ('아니', 0.011439160443842411), ('그', 0.011319410055875778), ('다', 0.010541032068431377), ('명', 0.009762655012309551), ('가', 0.009702779352664948), ('나', 0.0095530916005373), ('대선', 0.008924401365220547), ('지지', 0.008744776248931885), ('않', 0.008684900589287281)]
Top 10 words of topic #1
[('뽑', 0.02232348918914795), ('씨', 0.022131210193037987), ('표', 0.021554376929998398), ('여성', 0.020785264670848846), ('말', 0.019247040152549744), ('지', 0.01828564889729023), ('같', 0.01617058925330639), ('버리', 0.015016920864582062), ('나', 0.013670973479747772), ('보이', 0.013094139285385609)]
Top 10 words of topic #2
[('국민', 0.050977498292922974), ('힘', 0.042642269283533096), ('당', 0.03847465664148331), ('대선', 0.024266880005598068), ('명', 0.019909828901290894), ('대', 0.018962644040584564), ('국민의', 0.015363340266048908), ('선거', 0.015173903666436672), ('속보', 0.015173903666436672), ('윤석', 0.015173903666436672)]
Top 10 words of topic #3
[('뉴스', 0.0245045963

In [94]:
model.docs[4].get_topic_dist()

array([2.8831323e-03, 9.9416226e-01, 5.6221511e-04, 3.9142725e-04,
       4.3980026e-04, 2.4836941e-04, 3.8311863e-04, 3.0053416e-04,
       2.7590693e-04, 3.5319352e-04], dtype=float32)